In [15]:
# 4.6 serialization(직렬화, 데이터나 객체를 저장 가능한 형태 또는 전송 가능한 형태로 바꾸는 과정)
# chat 설정 저장하는 방법, 불러오는 방법

from langchain.chat_models import ChatOpenAI
from langchain.llms.openai import OpenAI
from langchain.llms.loading import load_llm

# chat = OpenAI(
#     temperature=0.1,
#     max_tokens=450,
#     model="gpt-4o-mini"
# )

# 설정 저장하기
# chat.save("model_gpt4_4o_mini.json")

# 설정 불러오기
chat = load_llm("model_gpt4_4o_mini.json")

chat.predict("Hi! I'm Sumin")

c:\Users\Sumin\Desktop\development\nomad_GPT\FULLSTACK-GPT\envGPT\Lib\site-packages\langchain\llms\openai.py:216: UserWarning: You are trying to use a chat model. This way of initializing it is no longer supported. Instead, please use: `from langchain.chat_models import ChatOpenAI`
  warnings.warn(
c:\Users\Sumin\Desktop\development\nomad_GPT\FULLSTACK-GPT\envGPT\Lib\site-packages\langchain\llms\openai.py:811: UserWarning: You are trying to use a chat model. This way of initializing it is no longer supported. Instead, please use: `from langchain.chat_models import ChatOpenAI`
  warnings.warn(


'Hi Sumin! How can I assist you today?'

In [ ]:

# 4.6 
# OpenAI를 사용할 때 비용 측정 방법
from langchain.chat_models import ChatOpenAI
from langchain.callbacks import get_openai_callback
from langchain.globals import set_debug

set_debug(False)

chat = ChatOpenAI(
    temperature=0.1,
)

with get_openai_callback() as usage:
    a = chat.predict("What is the recipe for sausage")
    b = chat.predict("What is the recipe for macaroon")
    print(a, b, "\n")
    print(usage)
    print("total cost:", usage.total_cost)
    print("total token:", usage.total_tokens)
    print("prompt tokens:", usage.prompt_tokens)
    print("completion tokens:", usage.completion_tokens)



In [ ]:

# 4.5 유용한 기능 소개
# set_llm_cache(InMemoryCache()): 답변을 저장함. 같은 질문을 받을 경우, 저장된 답변을 바로 출력함. 시스템을 껐다 켜면 사라짐.
# set_llm_cache(SQLiteCache("원하는이름.db")): 답변 저장. 파일로 저장하기 때문에 껐다 켜도 사용 가능.
# 이 외에도 다양한 cache를 사용할 수 있음. langchain에서는 다양한 third party를 제공하고 있음. 또한, 직접 cache를 만들어서 쓸 수도 있음.
# set_debug(True): llm에서 어떤 일이 일어나고 있는지 출력해줌. 답변이 나온 이유, 어떤 모델을 쓰고 있는지 등등.
from langchain.chat_models import ChatOpenAI
from langchain.callbacks import StreamingStdOutCallbackHandler
from langchain.globals import set_llm_cache, set_debug
from langchain.cache import InMemoryCache, SQLiteCache

# set_llm_cache(InMemoryCache())
set_llm_cache(SQLiteCache("cache.db"))
set_debug(True)

chat = ChatOpenAI(
    temperature=0.1,
    # streaming=True,
    # callbacks=[StreamingStdOutCallbackHandler(), ],
)

chat.predict("How do you make korea's Braised ribs")



In [ ]:

#4.4 메모리 합치기(많은 프롬프트를 하나로 합치기)
from langchain.chat_models import ChatOpenAI
from langchain.callbacks import StreamingStdOutCallbackHandler
from langchain.prompts import PromptTemplate
from langchain.prompts.pipeline import PipelinePromptTemplate

chat = ChatOpenAI(
    temperature=0.1,
    streaming=True,
    callbacks=[StreamingStdOutCallbackHandler(),],
)

intro = PromptTemplate.from_template(
    """
    You are a role playing assistant.
    And you are impersonating a {character}
"""
)

example = PromptTemplate.from_template(
    """
    This is an example of how you talk:

    Human: {example_question}
    You: {example_answer}
"""
)

start = PromptTemplate.from_template(
    """
    Start now!

    Human: {question}
    You:
"""
)

final = PromptTemplate.from_template(
    """
    {intro}

    {example}

    {start}
"""
)

prompts = [
    ("intro", intro),
    ("example", example),
    ("start", start),
]

full_prompt = PipelinePromptTemplate(final_prompt=final, pipeline_prompts=prompts)

# full_prompt.format(
#     character="Pirate",
#     example_question="What is your location?",
#     example_answer="Arrrg! That is a secret!",
#     question="What is your fav food?",
# )

chain = full_prompt | chat

# chain.invoke(
{
    "character":"Pirate",
    "example_question":"What is your location?",
    "example_answer":"Arrrg! That is a secret!",
    "question":"What is your fav food?",
}
)


In [ ]:

#4.4 파일에서 형식 불러오기
from langchain.chat_models import ChatOpenAI
from langchain.callbacks import StreamingStdOutCallbackHandler
from langchain.prompts import load_prompt

chat = ChatOpenAI(
    temperature=0.1,
    streaming=True,
    callbacks=[StreamingStdOutCallbackHandler(),],
)

# prompt = load_prompt("./prompt.json")
prompt = load_prompt("./prompt.yaml")
prompt.format(country="North Korea")


In [ ]:
# 4.3
from langchain.chat_models import ChatOpenAI
from langchain.prompts.few_shot import FewShotPromptTemplate
from langchain.callbacks import StreamingStdOutCallbackHandler
from langchain.prompts import PromptTemplate
from langchain.prompts.example_selector.base import BaseExampleSelector

chat = ChatOpenAI(
    temperature=0.1,
    streaming=True,
    callbacks=[StreamingStdOutCallbackHandler(), ]
)

examples = [
    {
        "question": "What do you know about France?",
        "answer": """
        Here is what I know:
        Capital: Paris
        Language: French
        Food: Wine and Cheese
        Currency: Euro
        """,
    },
    {
        "question": "What do you know about Italy?",
        "answer": """
        I Know this:
        Capital: Rome
        Language: Italian
        Food: Pizza and Pasta
        Currency: Euro
        """,
    },
    {
        "question": "what do you know about Greece?",
        "answer": """
        I Know this:
        Capital: Athens
        Language: Greek
        Food: Souvlaki and Feta Cheese
        Currency: Euro
        """,
    },
]

class RandomExampleSelector(BaseExampleSelector):

    def __init__(self, examples):
        self.examples = examples
    
    def add_example(self, example):
        self.examples.append(example)

    def select_examples(self, input_variables):
        from random import choice
        return [choice(self.examples)]

example_prompt = PromptTemplate.from_template("Human: {question}\nAI: {answer}")

example_selector = RandomExampleSelector(
    examples=examples,
)

prompt = FewShotPromptTemplate(
    example_prompt=example_prompt,
    example_selector=example_selector,
    suffix= "What do you know about {country}?", #형식화된 예제의 마지막에 나오는 내용
    input_variables=["country"]
)

prompt.format(country="Brazil")


In [ ]:

from langchain.chat_models import ChatOpenAI
from langchain.prompts.few_shot import FewShotPromptTemplate
from langchain.callbacks import StreamingStdOutCallbackHandler
from langchain.prompts import PromptTemplate
from langchain.prompts.example_selector import LengthBasedExampleSelector

chat = ChatOpenAI(
    temperature=0.1,
    streaming=True,
    callbacks=[StreamingStdOutCallbackHandler(), ]
)

examples = [
    {
        "question": "What do you know about France?",
        "answer": """
        Here is what I know:
        Capital: Paris
        Language: French
        Food: Wine and Cheese
        Currency: Euro
        """,
    },
    {
        "question": "What do you know about Italy?",
        "answer": """
        I Know this:
        Capital: Rome
        Language: Italian
        Food: Pizza and Pasta
        Currency: Euro
        """,
    },
    {
        "question": "what do you know about Greece?",
        "answer": """
        I Know this:
        Capital: Athens
        Language: Greek
        Food: Souvlaki and Feta Cheese
        Currency: Euro
        """,
    },
]

example_prompt = PromptTemplate.from_template("Human: {question}\nAI: {answer}")

example_selector = LengthBasedExampleSelector(
    examples=examples,
    example_prompt=example_prompt,
    max_length=180,
)

prompt = FewShotPromptTemplate(
    example_prompt=example_prompt,
    example_selector=example_selector,
    suffix= "What do you know about {country}?", #형식화된 예제의 마지막에 나오는 내용
    input_variables=["country"]
)

prompt.format(country="Brazil")

In [2]:

from langchain.chat_models import ChatOpenAI
from langchain.prompts.few_shot import FewShotChatMessagePromptTemplate
from langchain.callbacks import StreamingStdOutCallbackHandler
from langchain.prompts import ChatPromptTemplate


chat = ChatOpenAI(
    temperature=0.1,
    streaming=True,
    callbacks=[StreamingStdOutCallbackHandler(), ]
)

examples = [
    {
        "country": "France",
        "answer": """
        Here is what I know:
        Capital: Paris
        Language: French
        Food: Wine and Cheese
        Currency: Euro
        """,
    },
    {
        "country": "Italy",
        "answer": """
        I Know this:
        Capital: Rome
        Language: Italian
        Food: Pizza and Pasta
        Currency: Euro
        """,
    },
    {
        "country": "Greece",
        "answer": """
        I Know this:
        Capital: Athens
        Language: Greek
        Food: Souvlaki and Feta Cheese
        Currency: Euro
        """,
    },
]

example_prompt = ChatPromptTemplate.from_messages([
    ("human", "What do you know about {country}?"),
    ("ai", "{answer}")
])

example_prompt = FewShotChatMessagePromptTemplate(
    example_prompt=example_prompt,
    examples=examples,
)

final_prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a geography expert, you give short answer."),
    example_prompt,
    ("human", "What do you know about {country}?")
])

print(final_prompt)

chain = final_prompt | chat

chain.invoke({
    "country":"Thailand",
})



input_variables=['country'] messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], template='You are a geography expert, you give short answer.')), FewShotChatMessagePromptTemplate(examples=[{'country': 'France', 'answer': '\n        Here is what I know:\n        Capital: Paris\n        Language: French\n        Food: Wine and Cheese\n        Currency: Euro\n        '}, {'country': 'Italy', 'answer': '\n        I Know this:\n        Capital: Rome\n        Language: Italian\n        Food: Pizza and Pasta\n        Currency: Euro\n        '}, {'country': 'Greece', 'answer': '\n        I Know this:\n        Capital: Athens\n        Language: Greek\n        Food: Souvlaki and Feta Cheese\n        Currency: Euro\n        '}], example_prompt=ChatPromptTemplate(input_variables=['answer', 'country'], messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['country'], template='What do you know about {country}?')), AIMessagePromptTemplate(prompt=Promp

AIMessageChunk(content='\n        I Know this:\n        Capital: Bangkok\n        Language: Thai\n        Food: Pad Thai and Tom Yum\n        Currency: Thai Baht\n        ')

In [ ]:
# FewShotPromptTemplate, PromptTemplate
from langchain.chat_models import ChatOpenAI
from langchain.prompts.few_shot import FewShotPromptTemplate
from langchain.callbacks import StreamingStdOutCallbackHandler
from langchain.prompts import PromptTemplate

chat = ChatOpenAI(
    temperature=0.1,
    streaming=True,
    callbacks=[StreamingStdOutCallbackHandler(), ]
)
# chat.predict("What do you know about India?")


examples = [
    {
        "question": "What do you know about France?",
        "answer": """
        Here is what I know:
        Capital: Paris
        Language: French
        Food: Wine and Cheese
        Currency: Euro
        """,
    },
    {
        "question": "What do you know about Italy?",
        "answer": """
        I Know this:
        Capital: Rome
        Language: Italian
        Food: Pizza and Pasta
        Currency: Euro
        """,
    },
    {
        "question": "what do you know about Greece?",
        "answer": """
        I Know this:
        Capital: Athens
        Language: Greek
        Food: Souvlaki and Feta Cheese
        Currency: Euro
        """,
    },
]

# 1. 예제 형식 지정하기(좀 더 명확하게)
# example_template = """
#     Human: {question}
#     AI: {answer}
# """

# example_prompt = PromptTemplate.from_template(example_template)

# 2. 예제 형식 지정하기(간단)
example_prompt = PromptTemplate.from_template("Human: {question}\nAI: {answer}")

prompt = FewShotPromptTemplate(
    example_prompt=example_prompt,
    examples=examples, #example_prompt 와 examplaes를 가져와 형식화(formatting)함. 
    suffix= "What do you know about {country}?", #형식화된 예제의 마지막에 나오는 내용
    input_variables=["country"]
)

chain = prompt | chat

chain.invoke({
    "country":"Turkey",
})

In [ ]:
from langchain.prompts import PromptTemplate

# PromptTemplate Using #1 ======================
# t = PromptTemplate.from_template("What is the capital of {country}")
# t.format(country="India")

# PromptTemplate Using #2 ======================
# t = PromptTemplate(
#     template="What is the capital of {country}",
#     input_variables=["country"],
# )
# t.format(country="India")


In [ ]:
# ChatPromptTemplate, chain
from langchain.chat_models import ChatOpenAI
from langchain.callbacks import StreamingStdOutCallbackHandler
from langchain.prompts import ChatPromptTemplate

chat = ChatOpenAI(
    temperature=0.1,
    streaming=True,
    callbacks=[StreamingStdOutCallbackHandler(), ]
)

poet_template = ChatPromptTemplate.from_messages([
    ("system", "You are a poet. You don’t write anything other than poetry."),
    ("human", "Write me a poem. The theme of the poem is {language}.")
])

poet_chain = poet_template | chat

explain_template = ChatPromptTemplate.from_messages([
    ("system", "You are a critic. Read the poem and evaluate what the poem is like."),
    ("human", "{poem}")
])

explain_chain = explain_template | chat

final_chain = {"poem": poet_chain} | explain_chain

final_chain.invoke({
    "language":"javascript"
})

In [4]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate

chat = ChatOpenAI(
    temperature=0.1
)

template = ChatPromptTemplate.from_messages([
    ("system", "You are geography expert. And you only reply in {language}."),
    ("ai", "안녕, 나는 {name}!"),
    ("human", "What is the distance between {country_a} and {country_b}. Also, What is your name?")
])

prompt = template.format_messages(
    language="Korean",
    name="Sumin",
    country_a="India",
    country_b="Vietnam"
)

chat.predict_messages   (prompt)

AIMessage(content='인도와 베트남 사이의 거리는 약 3,500km입니다. 제 이름은 수민이에요.')

In [2]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts importPromptTemplate

chat = ChatOpenAI(
    temperature=0.1
)

template = PromptTemplate.from_template(
    "What is the distance between {country_a} and {country_b}?"
)

prompt = template.format(country_a="seoul", country_b="France")

chat.predict(prompt)

'The distance between Seoul, South Korea and France varies depending on the specific cities being compared. For example, the distance between Seoul and Paris, France is approximately 9,500 kilometers (5,900 miles) when measured in a straight line. However, the actual distance traveled by air or land would be longer due to the curvature of the Earth and the specific flight path taken.'

In [1]:
from langchain.chat_models import ChatOpenAI

chat = ChatOpenAI(
    temperature=0.1
)

from langchain.schema import HumanMessage, AIMessage, SystemMessage

messages = [
    SystemMessage(content="You are specialist cooker"),
    AIMessage(content="Hi, I'm Sumin"),
    HumanMessage(content="What is the receipe of onion soup?")
]

chat.predict_messages(messages)

AIMessage(content='Here is a simple recipe for French Onion Soup:\n\nIngredients:\n- 4 large onions, thinly sliced\n- 4 tablespoons butter\n- 2 tablespoons olive oil\n- 2 cloves garlic, minced\n- 1 teaspoon sugar\n- 1/2 cup dry white wine\n- 6 cups beef broth\n- Salt and pepper to taste\n- Baguette slices\n- Gruyere cheese, grated\n\nInstructions:\n1. In a large pot, melt the butter and olive oil over medium heat. Add the sliced onions and cook, stirring occasionally, until caramelized and golden brown, about 30-40 minutes.\n2. Add the minced garlic and sugar to the onions and cook for another 2-3 minutes.\n3. Pour in the white wine and cook for a few minutes to deglaze the pot.\n4. Add the beef broth to the pot and bring to a simmer. Season with salt and pepper to taste.\n5. Preheat the broiler. Ladle the soup into oven-safe bowls and top each with a slice of baguette and a generous amount of grated Gruyere cheese.\n6. Place the bowls under the broiler until the cheese is melted and b

In [9]:
# 5.0 ConversationBufferMemory
# 대화를 모두 저장
from langchain.memory import ConversationBufferMemory

memory = ConversationBufferMemory(return_messages=False)
#return_message: True(Chat모델 return 해줌.) / False(string 형태로 return 해줌.)

memory.save_context({"input":"Hi!"}, {"output":"How are you"})

memory.load_memory_variables({})

memory.save_context({"input":"Hi!"}, {"output":"How are you2"})

memory.load_memory_variables({})

memory.save_context({"input":"Hi!"}, {"output":"How are you3"})

memory.load_memory_variables({})

{'history': 'Human: Hi!\nAI: How are you\nHuman: Hi!\nAI: How are you2\nHuman: Hi!\nAI: How are you3'}

In [13]:
# 5.1 ConversationBufferWindowMemory
# 특정 개수까지 대화를 저장
from langchain.memory import ConversationBufferWindowMemory

memory = ConversationBufferWindowMemory(
    return_messages=True, # 챗모델 형태로 메시지를 전달해줌.
    k=4 # 4개까지 저장함.
)

def add_message(input, output):
    memory.save_context({"input": input}, {"output": output})

add_message(1,1)
add_message(2,2)
add_message(3,3)
add_message(4,4)
add_message(5,5)

memory.load_memory_variables({})

{'history': [HumanMessage(content='2'),
  AIMessage(content='2'),
  HumanMessage(content='3'),
  AIMessage(content='3'),
  HumanMessage(content='4'),
  AIMessage(content='4'),
  HumanMessage(content='5'),
  AIMessage(content='5')]}

In [1]:
from langchain.memory import ConversationBufferMemory
from langchain.chat_models import ChatOpenAI
from langchain.schema.runnable import RunnablePassthrough
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder

llm = ChatOpenAI(
  temperature=0.1,
  model="gpt-4o-mini",
  # verbose=False #Chat 내역을 보여줌.
)

memory = ConversationBufferMemory(
  llm=llm,
  return_messages=True,
)

template = ChatPromptTemplate.from_messages(
  [
    ("system", "You are a helpful AI talking to a human"),
    MessagesPlaceholder(variable_name="history"), #history로 들어온 값을 모두 받기 위해
    ("human", "{question}")
  ]
)

print(template)

def load_memory(_):
  print("==history==")
  print(memory.load_memory_variables({})["history"])
  return memory.load_memory_variables({})["history"] #history로 저장된 값을 

chain = RunnablePassthrough.assign(history=load_memory) | template | llm

def invoke_chain(question):
  result = chain.invoke({"question":question})
  memory.save_context(
    {"input":question},
    {"output": result.content},
  )
  print(result)
  
invoke_chain("Hi, I'm Sumin")
invoke_chain("I live in Jinju, Korea")
invoke_chain("What is my name")

print(memory.load_memory_variables({}))

input_variables=['history', 'question'] input_types={'history': typing.List[typing.Union[langchain.schema.messages.AIMessage, langchain.schema.messages.HumanMessage, langchain.schema.messages.ChatMessage, langchain.schema.messages.SystemMessage, langchain.schema.messages.FunctionMessage]]} messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], template='You are a helpful AI talking to a human')), MessagesPlaceholder(variable_name='history'), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['question'], template='{question}'))]
==history==
[]
content='Hi Sumin! How can I assist you today?'
==history==
[HumanMessage(content="Hi, I'm Sumin"), AIMessage(content='Hi Sumin! How can I assist you today?')]
content='That’s great! Jinju is known for its beautiful scenery and rich history, especially the Jinju Fortress and the Lantern Festival. Do you have any favorite places or activities in Jinju?'
==history==
[HumanMessage(content="Hi, I'm Sumin"), AIM

In [16]:
# 6.1 Data loader and Splitter
# loader(text, pdf, unstructured)
# Splitter()
from langchain.chat_models import ChatOpenAI
from langchain.document_loaders import TextLoader, PyPDFLoader, UnstructuredFileLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter, CharacterTextSplitter

# splitter #1
# splitter = RecursiveCharacterTextSplitter(
#     chunk_size=200, # 몇 글자를 기준으로 split 할 것인지
#     chunk_overlap=50, # 앞의 글자를 얼마나 가져올 것인지
# )

# splitter #2
# 위와의 차이는 자르는 character를 지정 가능.
splitter = CharacterTextSplitter(
    separator="\n",
    chunk_size=600,
    chunk_overlap=100,
)

loader = UnstructuredFileLoader("./files/chapter_one.docx")

# splitter 이용 1
# docs = loader.load()
# splitter.split_documents(docs)

# splitter 이용 2
loader.load_and_split(text_splitter=splitter)


Created a chunk of size 963, which is longer than the specified 600
Created a chunk of size 774, which is longer than the specified 600
Created a chunk of size 954, which is longer than the specified 600
Created a chunk of size 922, which is longer than the specified 600
Created a chunk of size 1168, which is longer than the specified 600
Created a chunk of size 821, which is longer than the specified 600
Created a chunk of size 700, which is longer than the specified 600
Created a chunk of size 745, which is longer than the specified 600
Created a chunk of size 735, which is longer than the specified 600
Created a chunk of size 1110, which is longer than the specified 600
Created a chunk of size 991, which is longer than the specified 600
Created a chunk of size 990, which is longer than the specified 600
Created a chunk of size 1741, which is longer than the specified 600
Created a chunk of size 2001, which is longer than the specified 600
Created a chunk of size 1900, which is longe

39